In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

In [2]:
def cargar_datos(ruta):
    df = pd.read_csv(ruta)
    return df

In [3]:
def limpiar_datos(df):
    df = df.drop_duplicates()
    df = df.dropna(subset=['feature1', 'feature2'])  # Ajusta columnas según tu dataset
    return df

In [4]:
def transformar_datos(df, scaler=None):
    df['date'] = pd.to_datetime(df['date'])
    df['feature1_log'] = np.log1p(df['feature1'])
    if scaler is None:
        scaler = StandardScaler()
        df[['feature1_scaled', 'feature2_scaled']] = scaler.fit_transform(df[['feature1', 'feature2']])
    else:
        df[['feature1_scaled', 'feature2_scaled']] = scaler.transform(df[['feature1', 'feature2']])
    return df, scaler

In [5]:
def entrenar_modelo(df):
    X = df[['feature1_scaled', 'feature2_scaled']]
    y = df['target']
    modelo = XGBClassifier(eval_metric='logloss', random_state=42)
    modelo.fit(X, y)
    return modelo

In [6]:
def guardar_modelo(modelo, scaler, ruta_modelo, ruta_scaler):
    joblib.dump(modelo, ruta_modelo)
    joblib.dump(scaler, ruta_scaler)

In [7]:
def cargar_modelo(ruta_modelo, ruta_scaler):
    modelo = joblib.load(ruta_modelo)
    scaler = joblib.load(ruta_scaler)
    return modelo, scaler

In [8]:
def predecir(modelo, scaler, nuevos_datos_csv):
    df_nuevos = pd.read_csv(nuevos_datos_csv)
    df_nuevos = limpiar_datos(df_nuevos)
    df_nuevos['date'] = pd.to_datetime(df_nuevos['date'])
    df_nuevos['feature1_log'] = np.log1p(df_nuevos['feature1'])
    df_nuevos[['feature1_scaled', 'feature2_scaled']] = scaler.transform(df_nuevos[['feature1', 'feature2']])
    X_nuevos = df_nuevos[['feature1_scaled', 'feature2_scaled']]
    preds = modelo.predict(X_nuevos)
    return preds

In [ ]:
if __name__ == "__main__":
    # Ruta a tus archivos CSV - Modificar en produccion
    ruta_datos = 'datos_financieros.csv'
    ruta_nuevos_datos = 'nuevos_datos.csv'
    ruta_modelo = 'modelo_financiero.pkl'
    ruta_scaler = 'scaler_financiero.pkl'

    # Carga y preparación de datos
    df = cargar_datos(ruta_datos)
    df = limpiar_datos(df)
    df, scaler = transformar_datos(df)

    # Entrenamiento y guardado del modelo
    modelo = entrenar_modelo(df)
    guardar_modelo(modelo, scaler, ruta_modelo, ruta_scaler)
    print("Modelo entrenado y guardado correctamente.")

    # Predicción con nuevos datos
    preds = predecir(modelo, scaler, ruta_nuevos_datos)
    print("Predicciones en nuevos datos:", preds)